In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
df = pd.read_csv("./airline_satisfaction.csv")
df.drop('Unnamed: 0', inplace = True, axis=1)
df.shape

(129880, 24)

In [3]:
df=df.dropna()

In [4]:
ordinal = ["Departure/Arrival time convenient","Ease of Online booking","Gate location",
           "Food and drink","Online boarding","Seat comfort","Inflight entertainment","On-board service","Leg room service",
           "Baggage handling","Checkin service","Inflight service","Cleanliness"]
df["Note_totale"]=df["Inflight wifi service"]
for i in ordinal:
    df["Note_totale"]+=df[i]

In [5]:
ordinal += ["Note_totale", "Inflight wifi service"]

In [6]:
def tranche_age(x):
    if x<18: return 0
    if x< 30: return 1
    if x< 50: return 2
    return 3
df["tranche_age"]=df.Age.apply(lambda x: tranche_age(x))

In [7]:
df["retard_en_vol"]=df["Arrival Delay in Minutes"]-df["Departure Delay in Minutes"]
#si negatif : le vol à rattraper du retard

In [8]:
mask = df["Arrival Delay in Minutes"] <0
mask.value_counts()
#aucun vol arrive en avance

False    129487
Name: Arrival Delay in Minutes, dtype: int64

In [9]:
target=["satisfaction"]
quantitative =["Age","Flight Distance","Arrival Delay in Minutes","Departure Delay in Minutes","retard_en_vol"]
ordinal+=["Class"]
qualitative=["Gender","Customer Type","Type of Travel","tranche_age"]

In [10]:
features=quantitative+qualitative+ordinal

In [ ]:
y=df.satisfaction.values
X=df[qualitative+quantitative].values
X_quantitative=df[quantitative].values
X_categorical=df[qualitative].values

In [ ]:
#One-hot Encoding pour les variables qualitative
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder=OneHotEncoder()
one_hot_encoder.fit(X_categorical)
one_hot_encoded_X=one_hot_encoder.transform(X_categorical)

In [ ]:
one_hot_encoded_X.toarray()
one_hot_encoder.get_feature_names_out()

print("nombre de variables")
len(one_hot_encoder.get_feature_names_out())-2

In [ ]:
X_encoded = np.concatenate([one_hot_encoded_X.toarray(), X_quantitative], axis=1)

In [ ]:
print(f"Initial number of features: {X_quantitative.shape[1] + X_categorical.shape[1]}")
print(f"New number of features: {X_encoded.shape[1]}")

In [ ]:
#ORDINAL ENCODING
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()
X_ordinal=df[ordinal].values

In [ ]:
encoded_vars = []
for categorical_var in X_ordinal.T:
    encoded_vars.append(label_encoder.fit_transform(categorical_var))
    print(f"===== Encoded classes: {label_encoder.classes_}")

In [ ]:
X_categorical = np.array(encoded_vars).T

In [ ]:
X_labelled_categories=np.concatenate([encoded_vars],axis=1)
X_labelled=np.concatenate([X_labelled_categories.T, X_quantitative],axis=1)
X_labelled

In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
min_max_scaler.fit(X_quantitative)
scaled_quantitative = min_max_scaler.transform(X_quantitative)

In [ ]:
############

In [22]:
from sklearn.dummy import DummyClassifier

In [25]:
y=df[target].values
X=df[quantitative]

In [26]:
# I create an object of class DummyClassifier
dummy_classifier = DummyClassifier(strategy="constant", constant=0)

# I train it
dummy_classifier.fit(X, y)

ValueError: The constant target value must be present in the training data. You provided constant=0. Possible values are: ['neutral or dissatisfied', 'satisfied'].

In [ ]:
##########
#A KNeighborsClassifier with and
"""
Bonus: Test other distances, and be critical about it!, especially if you have qualitative variable.

Bonus++: Create your own distance, especially if you're working with a mix of quantitative/qualitative/ordinal variables.

Plot on the same plot the accuracy score, the recall score, the precision score, as a function of

for each distance (and put them side by side).

"Conclude" regarding:

    The performance of knn in general on your dataset
    The best 

    to select
    The best distance to select

Bonus: perform a deeper analysis of your classifier, using graphical and statistical tools to analyze the records that were misclassified, and try explaining the results.
"""

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score

In [ ]:
accuracy_scores_euc = []
precision_scores_euc = []
recall_scores_euc = []

for k in [1, 3, 5, 7, 9]:
    print(f"k = {k}")
    knn = KNeighborsClassifier(n_neighbors=k, p=2)
    print("======= Train KNN")
    knn.fit(X, y)
    print("======= Predict on training data")
    prediction = knn.predict(X)
    accuracy_scores_euc.append(accuracy_score(prediction, y))
    precision_scores_euc.append(precision_score(prediction, y))
    recall_scores_euc.append(recall_score(prediction, y))
    
accuracy_scores_man = []
precision_scores_man = []
recall_scores_man = []

neighbors = [1, 3, 5, 7, 9]

for k in neighbors:
    print(f"k = {k}")
    knn = KNeighborsClassifier(n_neighbors=k, p=1)
    print("======= Train KNN")
    knn.fit(X, y)
    print("======= Predict on training data")
    prediction = knn.predict(X)
    accuracy_scores_man.append(accuracy_score(prediction, y))
    precision_scores_man.append(precision_score(prediction, y))
    recall_scores_man.append(recall_score(prediction, y))

In [ ]:
plt.plot(neighbors, accuracy_scores_euc)
plt.title("Evolution of the accuracy score as a function of the number of neighbors (dist = euclidean)")
plt.show()

plt.plot(neighbors, precision_scores_euc)
plt.title("Evolution of the precision score as a function of the number of neighbors (dist = euclidean)")

plt.show()

plt.plot(neighbors, recall_scores_euc)
plt.title("Evolution of the recall score as a function of the number of neighbors (dist = euclidean)")

plt.show()

In [ ]:
"""Splitting into train and test"""

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:


# Split intro 4 datasets: X_train, y_train, X_test, y_test, with a 70% / 30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)



In [ ]:
split_accuracy = []
for i in range(1, 600, 100):
    knn = KNeighborsClassifier(n_neighbors=i, p=2)
    knn.fit(X_train, y_train)
    print(f"k={i}")
    prediction = knn.predict(X_test)
    print(f"Accuracy: {accuracy_score(y_test, prediction)}")
    split_accuracy.append(accuracy_score(y_test, prediction))

In [ ]:
plt.plot(split_accuracy)

In [ ]:


knn_final = KNeighborsClassifier(n_neighbors=9, p=2)
knn_final.fit(X, y)
final_score = accuracy_score(knn_final.predict(X), y)

print("=====")
print(final_score)
print("=====")



In [ ]:
#Cross validation - k-fold cross validation

In [ ]:


from sklearn.model_selection import cross_val_score



In [ ]:


cross_val_accuracy = cross_val_score(estimator=dummy_classifier, X=X, y=y, scoring="accuracy")
cross_val_precision = cross_val_score(estimator=dummy_classifier, X=X, y=y, scoring="precision")



In [ ]:
print("========================================")
print(f"=== Average accuracy: {np.mean(cross_val_accuracy)}")
print(f"=== Median accuracy: {np.median(cross_val_accuracy)}")
print("========================================")
print("========================================")

print(f"=== Average precision: {np.mean(cross_val_precision)}")
print(f"=== Median precision: {np.median(cross_val_precision)}")
print("========================================")

In [ ]:
cross_val_accuracies = []
cross_val_accuracies_std = []
for k_neighbor in range(1, 15, 2):
    knn = KNeighborsClassifier(n_neighbors=k_neighbor)
    print("======== Score on fold")
    cross_val_accuracy = np.median(cross_val_score(estimator=knn, X=X, y=y, scoring="accuracy", cv=5))
    cross_val_accuracy_std = np.std(cross_val_score(estimator=knn, X=X, y=y, scoring="accuracy", cv=5))
    print(f"Standard error on fold: {cross_val_accuracy_std}")
    cross_val_accuracies.append(cross_val_accuracy)
    cross_val_accuracies_std.append(cross_val_accuracy_std)

In [ ]:


plt.plot(cross_val_accuracies)



In [ ]:
#Choix du k, de la feature le meilleur score, et itérativement jusqu'à 
#maximiser le score